In [59]:
import numpy as np
import matplotlib.pyplot as plt
import segyio
import matplotlib.patches as patches
import pandas as pd
import re
import os

In [60]:
def read_seismic_cube(file_path):
    """
    :param file_path: Путь к файлу
    :return: возвращает куб в виде списков по Inlines, Xlines и Sampels (координаты x, y, z)
    а также словарю сейсмотрасс, где ключ это координата пересечения Inlines и Xlines, а значение это список значений сейсмотрасс
    """
    with segyio.open(file_path, 'r') as segyfile:
        x = list(segyfile.ilines)
        y = list(segyfile.xlines)
        z = list(segyfile.samples)
        cube = segyio.tools.cube(segyfile)
        traces = {f"{x[i]}_{y[j]}": cube[i][j][:] for i in range(len(x)) for j in range(len(y))}
    return x, y, z, traces

In [61]:
file_path = 'C:/HV/Seismic/datas/Cube_TWT.segy'
ilines, xlines, samples_0, traces = read_seismic_cube(file_path)
samples = list()
for i in range(len(samples_0)):
    samples.append(int(samples_0[i]))

In [62]:
# проход по всем разломам и составление всех точек в один большой словарь
def process_files(directory):
    files = os.listdir(directory)
    result = {}

    for file in files:
        if file.endswith('.txt'):
            with open(os.path.join(directory, file), 'r') as f:
                data = []
                for line in f:
                    fields = line.split()
                    if len(fields) == 8:
                        x = int(fields[2])
                        y = int(fields[1])
                        z = -int(float(fields[5]))
                        t = int(fields[7])
                        data.append([x, y, z, t])
                result[file] = data

    return result

In [63]:
directory = 'C:/HV/Seismic/datas/Faults_TWT_Charisma_fault_sticks_ASCII'
faults_original = process_files(directory)


In [64]:

"""
Сборка координат в один список попарно
Если точек больше 2, то берется минимальное и максимальное значение по глубинам
Если точка 1, то она не учитывается в разломах
"""
faults_final = {}

for key in faults_original:
    dop_spisok = []
    z_spisok = []
    faults_final[key] = []

    for i in range(len(faults_original[key])):
        if len(dop_spisok) == 0:
            dop_spisok.append(faults_original[key][i])
            z_spisok.append(faults_original[key][i][2])
        elif len(dop_spisok) == 1:
            if dop_spisok[0][3] == faults_original[key][i][3]:
                dop_spisok.append(faults_original[key][i])
                z_spisok.append(faults_original[key][i][2])
            else:
                dop_spisok.clear()
                z_spisok.clear()
                dop_spisok.append(faults_original[key][i])
                z_spisok.append(faults_original[key][i][2])
        elif len(dop_spisok) == 2:
            if dop_spisok[0][3] == faults_original[key][i][3]:
                dop_spisok.append(faults_original[key][i])
                z_spisok.append(faults_original[key][i][2])
            else:
                #faults_final.append(dop_spisok[:])
                faults_final[key].append(dop_spisok[:])
                dop_spisok.clear()
                z_spisok.clear()
                dop_spisok.append(faults_original[key][i])
                z_spisok.append(faults_original[key][i][2])
        else:
            if dop_spisok[0][3] == faults_original[key][i][3]:
                dop_spisok.append(faults_original[key][i])
                z_spisok.append(faults_original[key][i][2])
            else:
                faults_final[key].append(
                    [dop_spisok[z_spisok.index(min(z_spisok))], dop_spisok[z_spisok.index(max(z_spisok))]])
                dop_spisok.clear()
                z_spisok.clear()
                dop_spisok.append(faults_original[key][i])
                z_spisok.append(faults_original[key][i][2])

In [65]:
def line_km(spisok):
    z_min = -2386
    z_max = -2142
    if (-2142 < spisok[0][2]) and (spisok[1][2] < -2386):
        t1 = (z_max - spisok[0][2]) / (spisok[1][2] - spisok[0][2])
        x1 = round(spisok[0][0] + (spisok[1][0] - spisok[0][0]) * t1)
        y1 = round(spisok[0][1] + (spisok[1][1] - spisok[0][1]) * t1)
        z1 = z_max

        t2 = (z_min - spisok[0][2]) / (spisok[1][2] - spisok[0][2])
        x2 = round(spisok[0][0] + (spisok[1][0] - spisok[0][0]) * t2)
        y2 = round(spisok[0][1] + (spisok[1][1] - spisok[0][1]) * t2)
        z2 = z_min
        return [[x1, y1, z1], [x2, y2, z2], 0]
    elif (-2142 > spisok[0][2]) and (spisok[1][2] < -2386):
        t2 = (z_min - spisok[0][2]) / (spisok[1][2] - spisok[0][2])
        x2 = round(spisok[0][0] + (spisok[1][0] - spisok[0][0]) * t2)
        y2 = round(spisok[0][1] + (spisok[1][1] - spisok[0][1]) * t2)
        z2 = z_min
        return [[spisok[0][0], spisok[0][1], spisok[0][2]], [x2, y2, z2], 1]
    elif (-2142 < spisok[0][2]) and (spisok[1][2] > -2386):
        t1 = (z_max - spisok[0][2]) / (spisok[1][2] - spisok[0][2])
        x1 = round(spisok[0][0] + (spisok[1][0] - spisok[0][0]) * t1)
        y1 = round(spisok[0][1] + (spisok[1][1] - spisok[0][1]) * t1)
        z1 = z_max
        return [[x1, y1, z1], [spisok[1][0], spisok[1][1], spisok[1][2]], 2]
    else:
        return [[spisok[0][0], spisok[0][1], spisok[0][2]], [spisok[1][0], spisok[1][1], spisok[1][2]], 3]

In [66]:
faults_to_cube_2 = {}
for key in faults_final:
    faults_to_cube_2[key] = []
    for i in range(len(faults_final[key])):
        faults_to_cube_2[key].append(line_km(sorted(faults_final[key][i], reverse=True, key=lambda xx: xx[2])))
maps_of_top = {}
for key in traces:
    maps_of_top[key] = traces[key][0]

In [22]:
def show_map(maps, k):
    # срез карты по осям ХУ

    # Извлечение данных для отрисовки
    points = []
    values = []
    for coord_str, value in maps.items():
        y, x = map(int, coord_str.split('_'))
        points.append((x, y))
        values.append(value)

    # Создание графика
    plt.figure(figsize=(6, 6))
    scatter = plt.scatter(x=list(zip(*points))[0], y=list(zip(*points))[1], c=values)
    plt.xticks([ilines[0], ilines[100], ilines[200], ilines[300], ilines[400], ilines[500], ilines[-1]])
    plt.yticks([xlines[0], xlines[100], xlines[200], xlines[300], xlines[400], xlines[500], xlines[-1]])
    plt.title('Карта')
    plt.ylabel('Инлайны')
    plt.xlabel('Кросслайны')
    plt.colorbar(scatter, label='Шкала значений сейсмотрассы')

    x_coords = []
    y_coords = []
    # проходимся по первому элементу в каждом подсписке и добавляем его координаты в список
    for key in faults_to_cube_2:
        x_coords = []
        y_coords = []
        for sublist in faults_to_cube_2[key]:
            x_coords.append(sublist[k][0])
            y_coords.append(sublist[k][1])
        plt.plot(x_coords, y_coords, color='red', linewidth=2)  # Линия красного цвета с толщиной 2

    plt.grid(False)  # Отключение сетки
    plt.show()






In [67]:
import plotly.graph_objects as go

def show_map_2(maps, k):
    # Извлечение данных для отрисовки
    points = []
    values = []
    for coord_str, value in maps.items():
        x, y = map(int, coord_str.split('_'))
        points.append((x, y))
        values.append(value)

    # Создание графика
    fig = go.Figure()

    # Добавляем рассеяние точек
    scatter = go.Scatter(
        x=list(zip(*points))[0],
        y=list(zip(*points))[1],
        mode='markers',
        marker=dict(
            size=10,
            color=values,
            colorscale='Viridis',  # Вы можете выбрать другую цветовую шкалу
            colorbar=dict(title='Шкала значений сейсмотрассы'),
            showscale=True
        ),
        name='Сейсмотрассы'
    )
    fig.add_trace(scatter)

    # Проходимся по первому элементу в каждом подсписке и добавляем его координаты в список
    for key in faults_to_cube_2:
        x_coords = []
        y_coords = []
        for sublist in faults_to_cube_2[key]:
            x_coords.append(sublist[k][0])
            y_coords.append(sublist[k][1])
        # Добавляем линии
        fig.add_trace(go.Scatter(
            x=x_coords,
            y=y_coords,
            mode='lines',
            line=dict(color='red', width=2),
            name='Линии разломов'
        ))

    # Настройка макета графика
    fig.update_layout(
        title='Карта',
        xaxis_title='Инлайны',
        yaxis_title='Кросслайны',
        showlegend=True
    )

    # Отображение графика
    fig.show()



In [68]:
show_map_2(maps_of_top, 0)
print(faults_to_cube_2)

razrezi = {}
for key in faults_to_cube_2:
    a = faults_to_cube_2[key][0][0][0]
    b = faults_to_cube_2[key][-1][0][0]
    razrezi[key] = a + (b - a) // 2

print(razrezi)

In [ ]:
def show_razrez(X, name):
    map_xlines_samples = {}
    for i in range(len(xlines)):
        for j in range(len(samples)):
            map_xlines_samples[f"{xlines[i]}_{samples[j]}"] = traces[f"{X}_{xlines[i]}"][j]

    # Извлечение данных для отрисовки
    points = []
    values = []
    for coord_str, value in map_xlines_samples.items():
        x, y = map(int, coord_str.split('_'))
        points.append((x, y * (-1)))
        values.append(value)

    # Создание графика
    plt.figure(figsize=(5, 5))
    scatter = plt.scatter(x=list(zip(*points))[0], y=list(zip(*points))[1], c=values)
    plt.xticks([xlines[0], xlines[100], xlines[200], xlines[300], xlines[400], xlines[500], xlines[-1]])
    plt.yticks([(-1) * samples[0], (-1) * samples[50], (-1) * samples[100], (-1) * samples[-1]])
    plt.title(name)
    plt.xlabel('Кросcлайны')
    plt.ylabel('Глубины')
    plt.colorbar(scatter, label='Шкала  значений сейсмотрассы')

    for key in faults_to_cube_2:
        x_coords = []
        y_coords = []
        for i in range(len(faults_to_cube_2[key])):
            for j in range(2):
                if (X - 3) <= faults_to_cube_2[key][i][j][0] <= (X + 3):
                    #if faults_to_cube_2[key][i][j][0] == X:
                    x_coords.append(faults_to_cube_2[key][i][j][1])
                    y_coords.append(faults_to_cube_2[key][i][j][2])
                    plt.plot(x_coords, y_coords, color='red', linewidth=2,
                             alpha=0.3)  # Линия красного цвета с толщиной 2

    plt.grid(False)  # Отключение сетки
    plt.show()

In [ ]:
for key in razrezi:
    show_razrez(razrezi[key], key)

In [56]:
import plotly.graph_objects as go

def show_map(maps, k):
    # Извлечение данных для отрисовки
    points = []
    values = []
    for coord_str, value in maps.items():
        x, y = map(int, coord_str.split('_'))
        points.append((x, y))
        values.append(value)

    # Создание графика
    fig = go.Figure()

    # Добавляем рассеяние точек
    scatter = go.Scatter(
        x=list(zip(*points))[0],
        y=list(zip(*points))[1],
        mode='markers',
        marker=dict(
            size=10,
            color=values,
            colorscale='Viridis',  # Вы можете выбрать другую цветовую шкалу
            colorbar=dict(title='Шкала значений сейсмотрассы'),
            showscale=True
        ),
        name='Сейсмотрассы'
    )
    fig.add_trace(scatter)

    # Проходимся по первому элементу в каждом подсписке и добавляем его координаты в список
    for key in faults_to_cube_2:
        x_coords = []
        y_coords = []
        for sublist in faults_to_cube_2[key]:
            x_coords.append(sublist[k][0])
            y_coords.append(sublist[k][1])
        # Добавляем линии
        fig.add_trace(go.Scatter(
            x=x_coords,
            y=y_coords,
            mode='lines',
            line=dict(color='red', width=2),
            name='Линии разломов'
        ))

    # Настройка макета графика
    fig.update_layout(
        title='Карта',
        xaxis_title='Инлайны',
        yaxis_title='Кросслайны',
        showlegend=True
    )

    # Отображение графика
    fig.show()


In [57]:
import plotly.graph_objects as go

def show_razrez(X, name):
    map_xlines_samples = {}
    for i in range(len(xlines)):
        for j in range(len(samples)):
            map_xlines_samples[f"{xlines[i]}_{samples[j]}"] = traces[f"{X}_{xlines[i]}"][j]

    # Извлечение данных для отрисовки
    points = []
    values = []
    for coord_str, value in map_xlines_samples.items():
        x, y = map(int, coord_str.split('_'))
        points.append((x, y * (-1)))  # Инвертируем y для глубины
        values.append(value)

    # Создание графика
    fig = go.Figure()

    # Добавляем рассеяние точек
    scatter = go.Scatter(
        x=list(zip(*points))[0],
        y=list(zip(*points))[1],
        mode='markers',
        marker=dict(
            size=10,
            color=values,
            colorscale='Viridis',  # Вы можете выбрать другую цветовую шкалу
            colorbar=dict(title='Шкала значений сейсмотрассы'),
            showscale=True
        ),
        name='Сейсмотрассы'
    )
    fig.add_trace(scatter)

    # Добавление линий разломов
    for key in faults_to_cube_2:
        x_coords = []
        y_coords = []
        for i in range(len(faults_to_cube_2[key])):
            for j in range(2):
                if (X - 3) <= faults_to_cube_2[key][i][j][0] <= (X + 3):
                    x_coords.append(faults_to_cube_2[key][i][j][1])
                    y_coords.append(faults_to_cube_2[key][i][j][2])
        if x_coords and y_coords:  # Проверяем, не пустые ли координаты
            fig.add_trace(go.Scatter(
                x=x_coords,
                y=y_coords,
                mode='lines',
                line=dict(color='red', width=2),  # Убираем opacity
                name='Линии разломов'
            ))

    # Настройка макета графика
    fig.update_layout(
        title=name,
        xaxis_title='Кросслайны',
        yaxis_title='Глубины',
        showlegend=True,
        yaxis=dict(autorange='reversed')  # Инвертируем ось Y для глубины
    )

    # Настройка меток по осям
    fig.update_xaxes(tickvals=[xlines[0], xlines[100], xlines[200], xlines[300], xlines[400], xlines[500], xlines[-1]])
    fig.update_yaxes(tickvals=[(-1) * samples[0], (-1) * samples[50], (-1) * samples[100], (-1) * samples[-1]])

    # Отображение графика
    fig.show()
